<a href="https://colab.research.google.com/github/kleinfossil/googlecolabfree_stable_diffusion/blob/main/ControlNet_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
ControlNet uses a conda environment. This is quite challenging to setup in google colab. 
It is not possible to setup and activete a conda environment via iphython. 
Instead it is required to run the environment in a bash shell. This again denies options like code highlighting and so on.
Condacolab supports an easy setup of a conda environment. However this works only for the base environment. Also it can not change the python version. 
As ControlNet requires an older python version, this requirement will be just ignored. 
The following will setup the CondaColab which will also restart the colab kernel. Therefore this cell need to be executed seperatly. 
If there is anybody who has a better idea let me know. 
"""
from IPython.display import clear_output
!pip install -q condacolab
import condacolab
condacolab.install()
clear_output()

In [ ]:
# Execution Time: 7m57s
# Disk Space after execution: 34.1 / 107.7

# Check if condacolab is installed correctly
from IPython.display import clear_output
import condacolab
condacolab.check()

# Download ControlNet Git
from IPython.display import clear_output
from IPython.utils import capture
import os

with capture.capture_output() as cap:
  if os.path.exists("/content/ControlNet"):
    %cd /content/ControlNet
    !git pull https://github.com/lllyasviel/ControlNet.git
  else:
      !git clone https://github.com/lllyasviel/ControlNet.git
print(cap)
clear_output()

"""
Adjust environment.yaml. This might become a point of failure.
If ther is any dependency error, then it is probably because of these changes.
"""
# Replace the environment name with base. condacolab works with base environment only.
!sed -i -e 's/name: control/name: base/g' /content/ControlNet/environment.yaml
# Delete the python requirement
!sed -i -e 's/  - python=3.8.5//g' /content/ControlNet/environment.yaml
# Delete cudatoolkit
!sed -i -e 's/  - cudatoolkit=11.3//g' /content/ControlNet/environment.yaml
# Clear empyt lines
!sed '/^$/d' /content/ControlNet/environment.yaml
clear_output()

"""
ControlNet does not support the share launch option out of the box.
A quick and dirty fix is to replace the launch call in the file.  
"""
!sed -i -e "s/block.lauch(server_name='0.0.0.0')/block.lauch(server_name='0.0.0.0', share=True)/g" /content/ControlNet/gradio_scribble2image.py

# Update the base environment and return error code
!mamba env update -f /content/ControlNet/environment.yaml
clear_output()

In [3]:
!pip install huggingface_hub
from huggingface_hub import hf_hub_download

def download_files():
  # annotator files
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/body_pose_model.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/dpt_hybrid-midas-501f0c75.pt", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/hand_pose_model.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/mlsd_large_512_fp32.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/mlsd_tiny_512_fp32.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/network-bsds500.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/upernet_global_small.pth", cache_dir="/content/ControlNet_Hugging_Downloads")

  # models
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename="models/control_sd15_scribble.pth", cache_dir="/content/ControlNet_Hugging_Downloads")


%mkdir -p /content/ControlNet_Hugging_Downloads
download_files()
clear_output()

In [ ]:
# Execution time: 1m 42s
from distutils.dir_util import copy_tree
copy_tree("/content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/annotator/", "/content/ControlNet/annotator/")
copy_tree("/content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/models/", "/content/ControlNet/models/")
clear_output()

In [ ]:
%cd /content/ControlNet/
!python gradio_scribble2image.py --share